In [1]:
import os
import pandas as pd
import pandas_gbq
from io import BytesIO
from google.cloud import storage
from google.oauth2 import service_account
from google.cloud import bigquery


In [36]:
def get_data_from_gbq(query):
    # Đường dẫn đến file JSON chứa thông tin xác thực
    credentials = service_account.Credentials.from_service_account_file('/home/admin1/Documents/App/airflow/dw-demo-435209-d671317c69dd.json')
    client = bigquery.Client(credentials=credentials)

    # Thực hiện truy vấn
    query_job = client.query(query)

    # Nhận kết quả
    results = query_job.result()

    # Chuyển đổi kết quả thành DataFrame
    return results.to_dataframe()


In [29]:
query = """
    WITH a as (
    SELECT 
    t1.user_id,
    min(t1.location) check_location,
    t1.check_date,
    min(t1.check_time) as checkin_time,
    max(t1.check_time) as checkout_time,
    t2.days_week,
    t2.shift_name,
    t2.check_in_shift_time,
    t2.check_out_shift_time
    FROM `dw-demo-435209.dwh_attendance.fact_attendance` t1
    LEFT JOIN `dw-demo-435209.dwh_attendance.attendance_result` t2 on t1.user_id = t2.user_id and t1.check_date = t2.date_check
    GROUP BY 
    t1.user_id, 
    t1.check_date,
    t2.days_week,
    t2.shift_name,
    t2.check_in_shift_time,
    t2.check_out_shift_time
    )
    SELECT 
    *,
    CASE WHEN checkin_time > check_in_shift_time THEN TIMESTAMP_DIFF(checkin_time, check_in_shift_time, MINUTE) ELSE NULL END AS so_phut_muon,
    CASE WHEN checkout_time < check_out_shift_time THEN TIMESTAMP_DIFF(check_out_shift_time, checkout_time, MINUTE) ELSE NULL END AS so_phut_ve_som,
    TIMESTAMP_DIFF(checkout_time, checkin_time, MINUTE) AS thoi_gian_lam_viec,
    TIMESTAMP_DIFF(check_out_shift_time, check_in_shift_time, MINUTE) AS thoi_gian_dinh_muc
    FROM a
"""
df = get_data_from_gbq(query)
df

/home/admin1/Documents/App/airflow/.venv/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,user_id,check_location,check_date,checkin_time,checkout_time,days_week,shift_name,check_in_shift_time,check_out_shift_time,so_phut_muon,so_phut_ve_som,thoi_gian_lam_viec,thoi_gian_dinh_muc
0,39178e1f,Ngõ Hoà Bình 2,2024-09-10,2024-09-10 08:48:05,2024-09-10 12:13:16,3,Ca_sáng,2024-09-10 08:30:00,2024-09-10 12:00:00,18,<NA>,205,210
1,39178e1f,None,2024-09-09,2024-09-09 08:23:42,2024-09-09 12:26:38,2,Ca_sáng,2024-09-09 08:30:00,2024-09-09 12:00:00,<NA>,<NA>,243,210
2,39178e1f,None,2024-09-07,2024-09-07 08:33:59,2024-09-07 12:07:54,7,Ca_sáng,2024-09-07 08:30:00,2024-09-07 12:00:00,3,<NA>,214,210
3,39178e1f,Công ty,2024-09-05,2024-09-05 11:57:52,2024-09-05 17:35:58,5,Ca_chiều,2024-09-05 13:00:00,2024-09-05 17:30:00,<NA>,<NA>,338,270
4,42a19972,Công ty,2024-09-16,2024-09-16 08:30:00,2024-09-16 17:34:23,2,8h30-17h30,2024-09-16 08:30:00,2024-09-16 17:30:00,<NA>,<NA>,544,540
